In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing liblaries for the algorithm

In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for plotting features

from skimage.feature import hog #importing hog feature
from skimage import data, feature, exposure
import cv2 #to read and modify images

Plotting an image sample using hog feature in the images

In [20]:
image = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/000001.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
fd, hog_image = hog(image, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(image, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()

extracting data

In [21]:
#importing data_attribute
data_atr = pd.read_csv("../input/celeba-dataset/list_attr_celeba.csv",index_col="image_id")
data_atr.head()

data cleaning

In [22]:
#clean data
data_atr.replace(to_replace=-1, value=0, inplace=True)
data_atr.head()

If NaN is present, fill it with a 0

In [23]:
#if NaN present fill a 0
data_atr = data_atr.fillna('0')
data_atr.info()

getting columns of attributes

In [24]:
#columns
print("ATTRIBUTES")
nUm = 1
for col in data_atr.columns:
    print(f"ATTRIBUTE {nUm} : {col}")
    nUm += 1

extracting Male attribute column

In [25]:
#identify gender using male column
gender = data_atr['Male']
gender.head(6)

Plotting a graph of gender distribution in the dataset

In [26]:
import seaborn as sns
# Gender distribution
plt.title('Gender Distribution')
sns.countplot(y = 'Male', data=data_atr)
plt.show()

extracting all, men and women image samples from the dataset and source

In [27]:
#load image id's with men and women attribute and fix them to an array
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

img_path = '../input/celeba-dataset/img_align_celeba/img_align_celeba/'
Men = data_atr.loc[data_atr['Male']==1]
men =Men.iloc[0:1000, : ]
mt = Men.iloc[100:2000, : ]
Women = data_atr.loc[data_atr['Male']==0]
women = Women.iloc[0:1000, : ]
wt = Women.iloc[100:2000, : ]
wt_i = np.array(wt.index)
mt_i = np.array(mt.index)
women_i = np.array(women.index) #holds women image names
men_i = np.array(men.index) # holds men image names
men_images = []
mt_images = []
wt_images =[]
women_images = []
for im in women_i:
    image = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+im)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    women_images.append(image)
    image = np.expand_dims(image, axis =0)
for im in men_i:
    image = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+im)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    men_images.append(image)
    image = np.expand_dims(image, axis =0)
for im in wt_i:
    image = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+im)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    wt_images.append(image)
for im in mt_i:
    image = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+im)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mt_images.append(image)
#all images
all_im = data_atr
im_all =all_im.iloc[1000:2000, : ]
all_i = np.array(im_all.index)
im_All = []
for im in all_i:
    image = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/'+im)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    im_All.append(image)

Training 

In [28]:
from itertools import chain
X_train = np.array([feature.hog(im,channel_axis=-1)
                    for im in chain(men_images,
                                    women_images)])
y_train = np.zeros(X_train.shape[0])
y_train[:(np.array(men_images)).shape[0]] = 1

In [29]:
y_train.shape

Training process

In [30]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

cross_val_score(GaussianNB(), X_train, y_train)


Getting the accuracy in percentage

In [49]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(LinearSVC(), {'C': [1.0, 2.0, 4.0, 8.0]})
grid.fit(X_train, y_train)
print("ACCURACY: ",(grid.best_score_)*100," %")


Getting the best model

In [32]:
grid.best_params_

Training the model again using the best model found above 

In [33]:
#retraining the whole algorithm using the best estimator
model = grid.best_estimator_
model.fit(X_train, y_train)

Defining elements that will make us know when the algorithm is full functioning

In [34]:
X_predict = np.array([feature.hog(im,channel_axis=-1)
                    for im in im_All])

Importing important liblaries for another section of training
Replacing 0 with Woman and 1 with Man in gender set

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X = np.array([feature.hog(im,channel_axis=-1)
                    for im in im_All ])
#clean data
gender.replace(to_replace=0, value="Female", inplace=True)
gender.replace(to_replace=1, value="Male", inplace=True)
y = gender.iloc[1000:2000] #taking gender variables for each image
gender.head()

Training and Testing session using LogisticRegression model

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)   
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#Model Building using logosticRegression
logit = make_pipeline(StandardScaler(), LogisticRegression())#removes default iteration number set
#Fitting algorithm
#X_test.reshape(1,-1)
#y_test.reshape(1,-1)
logit.fit(X_train, y_train)
#Calculating the acuracy
print("Accuracy ",(logit.score(X_test,y_test))*100,"%")
#y_test

Using the images array defined earlier we can predict the gender of each element using the algorithm

In [38]:
New_predict = logit.predict(X_predict)
print(New_predict)

Here the model accepts user inputs so that the gender can be predicted using the machine learning algorithm
Make sure the file path is correct

In [51]:
#allowing user inputs 
timage = cv2.imread('../input/celeba-dataset/img_align_celeba/img_align_celeba/040001.jpg')#change the filename to a desired one
timage = cv2.cvtColor(timage, cv2.COLOR_BGR2RGB)
fd, hog_image = hog(timage, orientations=8, pixels_per_cell=(16, 16),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=True)

ax1.axis('off')
ax1.imshow(timage, cmap=plt.cm.gray)
ax1.set_title('Input image')

# Rescale histogram for better display
hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))

ax2.axis('off')
ax2.imshow(hog_image_rescaled, cmap=plt.cm.gray)
ax2.set_title('Histogram of Oriented Gradients')
plt.show()
X_predict = np.array([feature.hog(timage,channel_axis=-1)])#defining the test image as the predict
New_predict = logit.predict(X_predict)
print(f'prediction: {New_predict} ')